# Lição 1 Demo 1: Criando uma Tabela com PostgreSQL


## Percorra os fundamentos do PostgreSQL:<br><li>Criando uma tabela, <li>Inserindo linhas de dados,<li>Executando uma consulta SQL simples para validar as informações. 

### Normalmente, usaríamos um wrapper python chamado psycopg2 para executar as consultas do PostgreSQL. Esta biblioteca deve ser pré-instalada, mas no futuro para instalar esta biblioteca, execute o seguinte comando no notebook para instalar localmente: 
!pip3 install --user psycopg2
#### Mais documentação pode ser encontrada aqui: http://initd.org/psycopg/ 

#### Importar a biblioteca 
Nota: Um erro pode aparecer após a execução deste comando. Leia-o com atenção antes de prosseguir.

In [1]:
import psycopg2

### Crie uma conexão com o banco de dados
1. Conecte-se à instância local do PostgreSQL ( 127.0.0.1 )
2. Use o banco de dados/schema da instância. 
3. A conexão alcança o banco de dados ( studentdb ) e usa os privilégios corretos para se conectar ao banco de dados ( user = postgres e password = 1234 ).

### Nota 1: Este bloco de código será padrão em todos os notebooks. 
### Nota 2: Adicionar o try except garantirá que os erros sejam detectados e compreendidos

In [3]:
try: 
    conn = psycopg2.connect("host=localhost port=5432 dbname=studentdb user=postgres password=1234")
except psycopg2.Error as e: 
    print("Error: Não foi possível conectar ao banco de dados Postgres")
    print(e)

### Use a conexão para obter um cursor que pode ser usado para executar consultas.

In [4]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Não foi possível colocar o cursor no banco de dados")
    print(e)

### Use o commit automático para que cada ação seja confirmada sem ter que chamar `conn.commit()` após cada comando. A capacidade de reverter e confirmar transações é um recurso dos Bancos de Dados Relacionais.

In [5]:
conn.set_session(autocommit=True)

### Teste a conexão e o código de tratamento de erros
O bloco try-except deve tratar o erro: Estamos tentando fazer um select * em uma tabela mas a tabela ainda não foi criada.

In [6]:
try: 
    cur.execute("select * from udacity.music_library")
except psycopg2.Error as e:
    print(e)

relation "udacity.music_library" does not exist
LINE 1: select * from udacity.music_library
                      ^



### Crie um banco de dados para trabalhar  

In [7]:
try: 
    cur.execute("create database udacity")
except psycopg2.Error as e:
    print(e)

### Feche nossa conexão com o banco de dados padrão, reconecte-se ao banco de dados Udacity e obtenha um novo cursor. 

In [10]:
try: 
    conn.close()
except psycopg2.Error as e:
    print(e)
  
try: 
    conn = psycopg2.connect("host=localhost port=5432 dbname=udacity user=postgres password=1234")
except psycopg2.Error as e: 
    print("Error: Não foi possível conectar ao banco de dados Postgres")
    print(e)
    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Não foi possível colocar o cursor no banco de dados")
    print(e)

conn.set_session(autocommit=True)

### Vamos criar uma biblioteca musical de álbuns. Cada álbum tem muitas informações que poderíamos adicionar à tabela da biblioteca de músicas. Começaremos com o nome do álbum, nome do artista, ano.  
`Table Name: music_library
column 1: Album Name
column 2: Artist Name
column 3: Year `


Revise este documento sobre tipos de dados PostgreSQL: https://www.postgresql.org/docs/9.5/datatype.html


In [11]:
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS music_library (album_name varchar, artist_name varchar, year int);")
except psycopg2.Error as e: 
    print("Error: Problema ao criar tabela")
    print (e)

### Nenhum erro foi encontrado, mas vamos verificar se nossa tabela foi criada. `select count(*)`que deve retornar 0, pois nenhuma linha foi inserida na tabela. 

In [12]:
try: 
    cur.execute("select count(*) from music_library")
except psycopg2.Error as e: 
    print("Error: Problema ao criar tabela")
    print (e)
    
print(cur.fetchall())

[(0,)]


### Inserir duas linhas

In [13]:
try: 
    cur.execute("INSERT INTO music_library (album_name, artist_name, year) \
                 VALUES (%s, %s, %s)", \
                 ("Let It Be", "The Beatles", 1970))
except psycopg2.Error as e: 
    print("Error: Não inseriu a linha")
    print (e)
    
try: 
    cur.execute("INSERT INTO music_library (album_name, artist_name, year) \
                 VALUES (%s, %s, %s)", \
                 ("Rubber Soul", "The Beatles", 1965))
except psycopg2.Error as e: 
    print("Error: Não inseriu a linha")
    print (e)

### Valide se seus dados foram inseridos na tabela. 
O loop while é usado para imprimir os resultados. Se estiver executando consultas no shell do Postgres, isso não será necessário.

### Observação: se você executar o código da instrução de inserção mais de uma vez, verá duplicatas de seus dados. O PostgreSQL permite duplicatas. ¶

In [14]:
try: 
    cur.execute("SELECT * FROM music_library;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('Let It Be', 'The Beatles', 1970)
('Rubber Soul', 'The Beatles', 1965)


### Como foi uma demostração iremos dar um Drop na tabela

In [15]:
try: 
    cur.execute("DROP table music_library")
except psycopg2.Error as e: 
    print("Error: Ao fazer Drop na tabela")
    print (e)

### Feche o cursor e a conexão

In [16]:
cur.close()
conn.close()